# ShortestDistanceSite example

In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen import Lattice
import numpy as np
import operator
from site_analysis import Atom, Analysis, ShortestDistanceSite, get_vertex_indices, AtomsTrajectory, SitesTrajectory
from collections import Counter
import tqdm

Load a `POSCAR` file where every octahedral site is occupied by a Na atom.

In [2]:
all_na_structure = Poscar.from_file('na_sn_all_na_ext.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'

Create a series of pymatgen Structures using the `Structure.from_spacegroup()` method, that each only contain the NaX sites, using the coordinates from Ramos _et al._ _Chem. Mater._ 2018.

In [3]:
sg = SpaceGroup('I41/acd:2')

In [4]:
from pymatgen import Structure, Lattice
lattice = all_na_structure.lattice
na1 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.25, 0.0, 0.125]])
na2 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.00, 0.0, 0.125]])
na3 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.25, 0.0]])
na4 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.0]])
na5 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.75, 0.25, 0.0]])
na6 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.5, 0.75, 0.625]])
i2  = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.666, 0.1376, 0.05]])
na_structures = {'Na1': na1,
                 'Na2': na2,
                 'Na3': na3,
                 'Na4': na4,
                 'Na5': na5,
                 'Na6': na6, 
                 'i2': i2}

In [5]:
na1_sites = [ ShortestDistanceSite(s.frac_coords, label='Na1') for s in na1 ]
na2_sites = [ ShortestDistanceSite(s.frac_coords, label='Na2') for s in na2 ]
na3_sites = [ ShortestDistanceSite(s.frac_coords, label='Na3') for s in na3 ]
na4_sites = [ ShortestDistanceSite(s.frac_coords, label='Na4') for s in na4 ]
na5_sites = [ ShortestDistanceSite(s.frac_coords, label='Na5') for s in na5 ]
na6_sites = [ ShortestDistanceSite(s.frac_coords, label='Na6') for s in na6 ]
i2_sites  = [ ShortestDistanceSite(s.frac_coords, label='i2') for s in i2 ]
sites = na1_sites + na2_sites + na3_sites + na4_sites + na5_sites + na6_sites + i2_sites

In [6]:
structure = Poscar.from_file('POSCAR').structure
# create Polyhedron objects
# create Atom objects
atoms = [Atom(species_string=centre_species) for site in structure if site.species_string is 'Na']
analysis = Analysis(sites, atoms)

In [7]:
print(analysis.site_labels())

['Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na1', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na3', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na4', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na5', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'Na6', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2', 'i2']


In [8]:
analysis.append_timestep(structure)

The occupations of each site are stored as a list of lists, as each site can have zero, one, or multiple atoms occupying it.

In [9]:
print(analysis.st[0])

[[84], [82], [85], [5], [75], [9], [78], [79], [83], [81], [], [88], [16], [73], [77], [80], [1], [76], [74], [12], [4], [14], [], [8], [15], [24], [21], [27], [17], [28], [19], [23], [2], [10], [7], [11], [3], [13], [6], [], [], [25], [20], [26], [86], [87], [18], [22], [57], [33], [29], [34], [47], [32], [30], [65], [], [51], [36], [40], [62], [38], [37], [39], [35], [67], [64], [70], [60], [72], [61], [], [68], [69], [71], [66], [58], [63], [59], [31], [48], [44], [46], [42], [56], [], [53], [50], [], [43], [45], [41], [55], [52], [54], [49], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [23]:
analysis.sites[0].frac_coords

array([0.25 , 0.   , 0.125])

In [24]:
[ s for s in structure if s.species_string=='Na'][83].frac_coords

array([0.28856436, 0.97768199, 0.13376162])

Rough example for collecting only occupied sites, and counting their site types:

In [10]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
c

Counter({'Na1': 15,
         'Na2': 29,
         'Na3': 15,
         'Na4': 15,
         'Na5': 14,
         'Na6': 0,
         'i2': 0})

vs. all sites:

In [11]:
c_sites = Counter(analysis.site_labels())
c_sites

Counter({'Na1': 16,
         'Na2': 32,
         'Na3': 16,
         'Na4': 16,
         'Na5': 16,
         'Na6': 8,
         'i2': 32})

In [12]:
analysis.reset()

xdatcar = Xdatcar('XDATCAR_Sn')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [13]:
n_timesteps = len(analysis.timesteps)
c_sites = Counter(analysis.site_labels())
c = Counter()
p_occ = {}
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for k, v in c.items():
    p_occ[k] = v / c_sites[k] / n_timesteps
p_occ

{'Na1': 0.93375,
 'Na2': 0.895625,
 'Na3': 0.915,
 'Na4': 0.9427083333333334,
 'Na5': 0.9120833333333334,
 'Na6': 0.0,
 'i2': 0.0026041666666666665}

In [14]:
# check total average occupation = 88 atoms
for k,v in c.items():
    print( k, p_occ[k]*c_sites[k])
print( sum( [ p_occ[k] * c_sites[k] for k, v in c.items()]))

Na1 14.94
Na2 28.66
Na3 14.64
Na4 15.083333333333334
Na5 14.593333333333334
Na6 0.0
i2 0.08333333333333333
88.0
